In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Virtual Try-On: Batch Generation Pipeline

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fuse-cases%2Fbatch_virtual_try_on.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/use-cases/batch_virtual_try_on.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/batch_virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>  

| Authors |
| --- |
| [Wafae Bakkali](https://github.com/WafaeBakkali) |
| [Katie Nguyen](https://github.com/katiemn) |

## Overview

### Virtual Try-On

[Virtual Try-On](https://cloud.google.com/vertex-ai/generative-ai/docs/image/generate-virtual-try-on-images) uses Google's advanced image generation models to create high-quality images of people virtually trying on clothing. By providing an image of a human model and a clothing product, you can generate an image of the model wearing that product. This technology is designed for application developers and retailers in the fashion industry.

This tutorial demonstrates how to use the Virtual Try-On API to automate batch processing, i.e., generating multiple results across various combinations of person and apparel images. In this tutorial, you will learn how to use the Google Gen AI SDK for Python to interact with the Virtual Try-On model to:
 - Upload images from local sources, Cloud Storage, or other URL-based inputs
 - Return side-by-side visual outputs to preview the try-on results

## Get started

### Install Google Gen AI SDK for Python

In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [3]:
import io
import time
import zipfile

from PIL import Image as PIL_Image
from google import genai
from google.colab import files
from google.genai.types import Image, ProductImage, RecontextImageSource
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
import requests

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Define helper functions

In [5]:
def display_images(images: list, type: str) -> None:
    fig, axes = plt.subplots(1, len(images), figsize=(12, 6))
    if len(images) == 1:
        axes = np.array([axes])
    for i, ax in enumerate(axes):
        if type == "local":
            image = img.imread(images[i])
        elif type == "url":
            response = requests.get(images[i])
            image = PIL_Image.open(io.BytesIO(response.content))
        # Display generated images
        else:
            image = images[i]._pil_image
        ax.imshow(image)
        ax.axis("off")
    plt.show()


def process_urls(url_string):
    urls = [url.strip() for url in url_string.split("\n") if url.strip()]
    return urls


def download_results_as_zip(results):
    zip_filename = f"virtual-try-on-results-{int(time.time())}.zip"
    with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zip_file:
        for i, result in enumerate(results):
            image_data = result.image_bytes
            zip_file.writestr(f"result_{i+1}.png", image_data)
    files.download(zip_filename)
    print(f"Download started for {zip_filename}")

### Load the image model

In [6]:
virtual_try_on = "virtual-try-on-preview-08-04"

## Batch processing with Virtual Try-On

### Upload person and apparel images

In this tutorial you can select your images from your local computer or public URLs. Choose only one option per batch processing job.

Supported Clothing:
  - **Tops:** shirts, hoodies, sweaters, tank tops, blouses
  - **Bottoms:** pants, leggings, shorts, skirts
  - **Footwear:** sneakers, boots, sandals, flats, heels, formal shoes

#### **Option 1:** Use local images

In this section, you'll upload images of people and clothing items to try them on from local files. Simply run the cell and select the images you'd like to use.

In [ ]:
print("Please upload one or more PERSON images:")
person_files = files.upload()
person_images = [img_name for img_name, img_bytes in person_files.items()]
display_images(person_images, "local")

print("\nPlease upload one or more PRODUCT images:")
product_files = files.upload()
product_images = [img_name for img_name, img_bytes in product_files.items()]
display_images(product_images, "local")

local_images = True

#### **Option 2:** Use public images

If you uploaded images through Option 1, skip this section. Otherwise, paste each URL on a new line in the appropriate variable below. Do not use comma-separated values or local file paths.

In [ ]:
# Paste person URLs here, one per line
person_urls = """
https://storage.googleapis.com/cloud-samples-data/generative-ai/image/man-in-field.png
https://storage.googleapis.com/cloud-samples-data/generative-ai/image/woman.jpg
"""

# Paste product URLs here, one per line
product_urls = """
https://storage.googleapis.com/cloud-samples-data/generative-ai/image/sweater.jpg
https://storage.googleapis.com/cloud-samples-data/generative-ai/image/trousers.jpg
"""

person_images = process_urls(person_urls)
product_images = process_urls(product_urls)

display_images(person_images, "url")
display_images(product_images, "url")

local_images = False

### Generate and view results

Running the cell below will generate an image for each combination of model and clothing item. The results will be displayed below.

In [ ]:
generated_results = []
total_jobs = len(person_images) * len(product_images)
current_job = 0

for person_img in person_images:
    for product_img in product_images:
        current_job += 1
        print(f"Running job {current_job}/{total_jobs}...")
        try:
            if local_images:
                person_image_obj = Image.from_file(location=person_img)
                product_image_obj = Image.from_file(location=product_img)
            else:
                person_response = requests.get(person_img)
                person_response.raise_for_status()
                person_image_obj = Image(image_bytes=person_response.content)

                product_response = requests.get(product_img)
                product_response.raise_for_status()
                product_image_obj = Image(image_bytes=product_response.content)

            generated_image = client.models.recontext_image(
                model=virtual_try_on,
                source=RecontextImageSource(
                    person_image=person_image_obj,
                    product_images=[ProductImage(product_image=product_image_obj)],
                ),
            )
            generated_results.append(generated_image.generated_images[0].image)
        except Exception as e:
            print(f"Skipping job {current_job}/{total_jobs} due to an error: {e}")

        time.sleep(2)  # Add a delay to help with API quota limits

# Display results
if generated_results:
    display_images(generated_results, "generated")

### Download results locally in a ZIP file

In [ ]:
if generated_results:
    download_results_as_zip(generated_results)